In [5]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import random

# ---------- 1. Generate Simulated Data ----------
def generate_synthetic_data(samples=200):
    # Simulate features like inlet_temp, feed_flow, air_pressure, etc.
    X = np.random.uniform(low=0, high=1, size=(samples, 4))
    # Output could represent yield, moisture content, etc.
    y = (0.3 * X[:, 0] + 0.5 * X[:, 1] + 0.2 * X[:, 2] + 0.4 * X[:, 3]) + np.random.normal(0, 0.05, size=samples)
    return train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = generate_synthetic_data()

# ---------- 2. Define Genetic Algorithm ----------
POP_SIZE = 10
GENS = 20

# Each individual is [hidden_layer_size, learning_rate_init, activation_function_index]
def create_individual():
    return [
        random.randint(5, 50),                          # neurons
        10**random.uniform(-4, -1),                    # learning rate
        random.choice([0, 1, 2])                       # activation: 0=identity, 1=logistic, 2=relu
    ]

def decode_activation(index):
    return ['identity', 'logistic', 'relu'][index]

def fitness(individual):
    hidden_layer_size = individual[0]
    learning_rate = individual[1]
    activation = decode_activation(individual[2])
    
    model = MLPRegressor(hidden_layer_sizes=(hidden_layer_size,), 
                         learning_rate_init=learning_rate,
                         activation=activation,
                         max_iter=500, random_state=1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    return -mse  # We want to minimize MSE

def selection(population, scores):
    sorted_pop = sorted(zip(population, scores), key=lambda x: x[1], reverse=True)
    return [ind for ind, score in sorted_pop[:POP_SIZE//2]]

def crossover(parent1, parent2):
    cut = random.randint(1, 2)
    child1 = parent1[:cut] + parent2[cut:]
    child2 = parent2[:cut] + parent1[cut:]
    return child1, child2

def mutate(individual):
    idx = random.randint(0, 2)
    if idx == 0:
        individual[0] = random.randint(5, 50)
    elif idx == 1:
        individual[1] = 10**random.uniform(-4, -1)
    else:
        individual[2] = random.choice([0, 1, 2])
    return individual

# ---------- 3. Run GA ----------
population = [create_individual() for _ in range(POP_SIZE)]

for gen in range(GENS):
    print(f"Generation {gen+1}")
    scores = [fitness(ind) for ind in population]
    print("Best score:", max(scores))
    selected = selection(population, scores)
    
    children = []
    while len(children) < POP_SIZE - len(selected):
        p1, p2 = random.sample(selected, 2)
        c1, c2 = crossover(p1, p2)
        children.extend([mutate(c1), mutate(c2)])
    
    population = selected + children[:POP_SIZE]

# ---------- 4. Get Best Individual ----------
best_individual = max(population, key=fitness)
print("\nBest Parameters Found:")
print("Hidden Layer Size:", best_individual[0])
print("Learning Rate:", best_individual[1])
print("Activation:", decode_activation(best_individual[2]))


Generation 1
Best score: -0.002488725787746017
Generation 2


/home/hp/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best score: -0.002488725787746017
Generation 3
Best score: -0.002488725787746017
Generation 4


/home/hp/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best score: -0.002488725787746017
Generation 5
Best score: -0.0023958079822043897
Generation 6
Best score: -0.0023958079822043897
Generation 7
Best score: -0.002327840745916017
Generation 8
Best score: -0.002327840745916017
Generation 9
Best score: -0.0022383880206898114
Generation 10
Best score: -0.0022383880206898114
Generation 11
Best score: -0.0022383880206898114
Generation 12
Best score: -0.0022383880206898114
Generation 13
Best score: -0.0022383880206898114
Generation 14
Best score: -0.0022383880206898114
Generation 15
Best score: -0.0022383880206898114
Generation 16
Best score: -0.0021931213364030934
Generation 17
Best score: -0.0021931213364030934
Generation 18
Best score: -0.0021931213364030934
Generation 19
Best score: -0.002028399145873974
Generation 20
Best score: -0.002028399145873974

Best Parameters Found:
Hidden Layer Size: 34
Learning Rate: 0.03395949142315878
Activation: relu
